In [1]:
import json

In [2]:
#!pip install docx2txt

In [3]:
#!pip install python-docx
#!pip install langdetect
#!pip install textblob
#!pip install guess_language-spirit

edits to original doc: philosophy.docx

- in word doc, go to Edit > Find > Advanced Find & Replace
- in "Find what" field, type: <[A-Za-z]
- select "Use Wildcards" and (at bottom) Format > Font > Italic
- in "Replace" field, type: ∞^&
- this should insert a "∞" before every italicized word
- & do the same for underlines

new file = philosophy emph.docx

In [167]:
import docx
from guess_language import guessLanguage
import re
import json

In [168]:
doc = docx.Document("philosophy emph.docx")
all_paras = doc.paragraphs
para_list=[]
for para in all_paras:
    text=para.text
    text=re.sub("∞","<i>",text)
    text=re.sub("¡","<u>",text)
    para_list.append(text)
para_text='\n'.join(para_list)

In [169]:
# split into sections
split_mark="¬"
para_text=re.sub("Ms-10",split_mark+"Ms-10",para_text) # insert split marker
sections0=re.split(split_mark,para_text)

sections=[]
for sec in sections0:
    if len(sec)>3: # incomplete sections?
        sec=re.sub("·","•",sec)
        #sec=re.sub("\(|\)","†",sec)
        sections.append(sec)
sections.sort()
print(len(sections))

1033


In [170]:
numdot="[0-9]•[0-9]+[^\t]*"
numbrack="[0-9]+\[\w+\]"
cr="[0-9]+[.][0-9]+[.][0-9]+ \([0-9,\s]+\)[*+]*"

# fields and their corresponding regular expressions- 
fields=[("manuscript","Ms-.*\(NB\)"), #Ms-101,12r[2] et 13r[1] (1914--0902) (NB)
         ("pt-number","\n\s*("+numdot+")\t"), #5•3063
         ("pt-page","\t("+numbrack+")\t"), #52[5]
        ("tlp-number","\t"+numbrack+"\t\s*("+numdot+")\t"),
         ("cross-reference","\t"+cr+"\n|\t"+cr+"[\s]+"+cr+"\n"), #22.8.14 (2)*  2.9.14 (1)**
         ("date","\n\s*([0-9]+[.][0-9]+[.][0-9]+[.]*)")] #2.9.14.
re.findall(fields[4][1],sections[0])

['\t22.8.14 (2)*  2.9.14 (1)**\n']

In [171]:
# grab into json
nodes=[]
empties=[]
multiples=[]
sos=[]
sections.sort()
for section in sections:
    text=[]
    
    # metadata
    s={}
    e=[]
    m=[]
    for field in fields:
        line=re.findall(field[1],section)
        if len(line)==1:
            s[field[0]]=line[0]
        elif len(line)==0:
            s[field[0]]=""
            e.append(s)
        elif len(line)>1:
            #s[field[0]]=line
            #OR
            for i in range(len(line)):
                s[field[0]+str(i+1)]=line[i]
            m.append(s)
    if len(e)in[2,3,4]:
        new=(section,e)
        empties.append(new)
    if m!=[]:
        new=(section,m[-1])
        multiples.append(new)
        
    # translations
    subsections=re.split("\n",section)
    ger=[]
    ger_ind=[]
    eng=[]
    eng_ind=[]
    for subsec in subsections:
        if (guessLanguage(subsec) == 'de'):
            ger_ind.append(subsections.index(subsec))
        elif (guessLanguage(subsec) == 'en'):
            eng_ind.append(subsections.index(subsec))
    if eng_ind!=[]:
        eng_start=sorted(eng_ind)[0]
        s["eng"]='\n'.join(subsections[eng_start:])
    else:
        eng_start=len(subsections)
        s["eng"]=""
    if ger_ind!=[]:
        ger_start=sorted(ger_ind)[0]
        s["ger"]='\n'.join(subsections[ger_start:eng_start])
    else:
        s["ger"]=""

    nodes.append(s)
        
print(len(nodes))
print(len(empties))
print(len(multiples))

<ipython-input-171-03ddc006a337>:41: DeprecationWarning: call to deprecated function guessLanguage()
  if (guessLanguage(subsec) == 'de'):
<ipython-input-171-03ddc006a337>:43: DeprecationWarning: call to deprecated function guessLanguage()
  elif (guessLanguage(subsec) == 'en'):


1033
181
22


In [172]:
with open('stern_hz.json', 'w', encoding='utf8') as f:
    json.dump(nodes, f, indent=4, ensure_ascii=False)

In [173]:
# View empties
for e in empties:
    print(e[0],'\n',e[1],'\n\n•••\n')

Ms-101,17r[2] (1914--0904) (NB)
 
4.9.14
Wenn nicht die Existenz des Subjekt-Prädikat <u>Satzes alles Nötige zeigt dann könnte es doch nur die Existenz irgend einer besonderen Tatsache jener Form zeigen. Und die Kenntnis einer solchen kann nicht für die Logik wesentlich sein.

If the existence of the subject-predicate <u>sentence does not show everything necessary, then it could surely only be shown by the existence of some specific fact of that form. And knowing such a fact cannot be essential for logic.

 		 
 
 [{'manuscript': 'Ms-101,17r[2] (1914--0904) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date': '4.9.14', 'eng': 'If the existence of the subject-predicate <u>sentence does not show everything necessary, then it could surely only be shown by the existence of some specific fact of that form. And knowing such a fact cannot be essential for logic.\n\n \t\t \n', 'ger': 'Wenn nicht die Existenz des Subjekt-Prädikat <u>Satzes alles Nötige zeigt d

 [{'manuscript': 'Ms-103,65r[3] (1916--1020) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date': '20.10.16.', 'eng': '', 'ger': 'Es ist klar daß mein Gesichtsraum der Länge nach anders beschaffen ist als der Breite nach. \n\n  \t\t \n'}, {'manuscript': 'Ms-103,65r[3] (1916--1020) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date': '20.10.16.', 'eng': '', 'ger': 'Es ist klar daß mein Gesichtsraum der Länge nach anders beschaffen ist als der Breite nach. \n\n  \t\t \n'}, {'manuscript': 'Ms-103,65r[3] (1916--1020) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date': '20.10.16.', 'eng': '', 'ger': 'Es ist klar daß mein Gesichtsraum der Länge nach anders beschaffen ist als der Breite nach. \n\n  \t\t \n'}, {'manuscript': 'Ms-103,65r[3] (1916--1020) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date': '20.10.16.', 'eng': '', 'ger': 'Es ist klar daß mei

In [174]:
# View multiples
for m in multiples:
    print(m[0],'\n',m[1],'\n\n\n')

Ms-101,31r[6] et 32r[1] (1914--1003) (NB)
 4•4482	45[5]	4•462 (1)	3.10.14 (4)  6.6.15 (4)
4•449	45[6]	4•465**		3.10.14 (4)  25.5.15 (4)

Tautologien sagen nichts aus, sie sind nicht Bilder von Sachverhalten: Sie sind / selber logisch vollkommen neutral. (Das logische Produkt einer Tautologie und eines Satzes sagt nicht mehr noch weniger aus als dieser allein.)

 Tautologies state nothing, they are are not pictures of states of things. They are themselves logically completely neutral. (The logical product of a tautology and a propostion says neither more nor less than the latter by itself.) [<i>See 4.462 and <i>cf. 4.465.]

		 
 
 {'manuscript': 'Ms-101,31r[6] et 32r[1] (1914--1003) (NB)', 'pt-number1': '4•4482', 'pt-number2': '4•449', 'pt-page1': '45[5]', 'pt-page2': '45[6]', 'tlp-number1': '4•462 (1)', 'tlp-number2': '4•465**', 'cross-reference1': '\t3.10.14 (4)  6.6.15 (4)\n', 'cross-reference2': '\t3.10.14 (4)  25.5.15 (4)\n', 'date': '', 'eng': ' Tautologies state nothing, they are

 {'manuscript': 'Ms-103,42r[5] (1916--0808? --0811?) (NB)', 'pt-number': '', 'pt-page': '', 'tlp-number': '', 'cross-reference': '', 'date1': '8.8.16.', 'date2': '11.8.16.', 'eng': '', 'ger': 'Jedem Gegenstand stehe ich objektiv gegenüber. Dem Ich nicht. \n\n\t\t \n'} 



Ms-103,79r[6] et 80r[1] (1916--1121) (NB)
 4•4303	46[6]	4•5 (3)b**		16.4.16 (1)+  21.11.16 (1)+
4•43012	78[11]	4•5 (3a) 		16.4.16 (1)+   21.11.16 (1)+

21.11.16.
Daß es möglich ist die allgemeine Satzform aufzustellen sagt nichts anderes als: Jede mögliche Satzform  / muß sich <u>voraussehen lassen. 

  		 
 
 {'manuscript': 'Ms-103,79r[6] et 80r[1] (1916--1121) (NB)', 'pt-number1': '4•4303', 'pt-number2': '4•43012', 'pt-page1': '46[6]', 'pt-page2': '78[11]', 'tlp-number1': '4•5 (3)b**', 'tlp-number2': '4•5 (3a) ', 'cross-reference1': '\t16.4.16 (1)+  21.11.16 (1)+\n', 'cross-reference2': '\t16.4.16 (1)+   21.11.16 (1)+\n', 'date': '21.11.16.', 'eng': '', 'ger': 'Daß es möglich ist die allgemeine Satzform aufzustellen